In [68]:
import pandas as pd

from datetime import datetime, timedelta

import numpy as np

import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import calendar

In [2]:
def get_klines_iter(symbol, interval, start, end = None, limit=1000):

    df = pd.DataFrame()

    if start is None:
        print('start time must not be None')
        return
    start = calendar.timegm(datetime.fromisoformat(start).timetuple()) * 1000

    if end is None:
        dt = datetime.now()
        utc_time = dt.replace()
        end = int(utc_time.timestamp()) * 1000
        return
    else:
        end = calendar.timegm(datetime.fromisoformat(end).timetuple()) * 1000
    last_time = None

    while len(df) == 0 or (last_time is not None and last_time < end):
        url = 'https://api.binance.com/api/v3/klines?symbol=' + \
              symbol + '&interval=' + interval + '&limit=1000'
        if(len(df) == 0):
            url += '&startTime=' + str(start)
        else:
            url += '&startTime=' + str(last_time)

        url += '&endTime=' + str(end)
        df2 = pd.read_json(url)
        df2.columns = ['Opentime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Closetime',
                       'Quote asset volume', 'Number of trades', 'Taker by base', 'Taker buy quote', 'Ignore']
        dftmp = pd.DataFrame()
        dftmp = pd.concat([df2, dftmp], axis=0, ignore_index=True, keys=None)

        dftmp.Opentime = pd.to_datetime(dftmp.Opentime, unit='ms')
        dftmp['Date'] = dftmp.Opentime.dt.strftime("%Y/%m/%d")
        dftmp['Time'] = dftmp.Opentime.dt.strftime("%H:%M:%S")
        dftmp = dftmp.drop(['Quote asset volume', 'Closetime', 'Opentime',
                      'Number of trades', 'Taker by base', 
                            'Taker buy quote', 'Ignore', 
                            "High", "Low", 
                            "Close", "Volume"], axis=1)
        
        column_names = ["Date", "Time", "Open"]
        dftmp.reset_index(drop=True, inplace=True)
        dftmp = dftmp.reindex(columns=column_names)
        string_dt = str(dftmp['Date'][len(dftmp) - 1]) + 'T' + str(dftmp['Time'][len(dftmp) - 1]) + '.000Z'
        utc_last_time = datetime.strptime(string_dt, "%Y/%m/%dT%H:%M:%S.%fZ")
        last_time = (utc_last_time - datetime(1970, 1, 1)) // timedelta(milliseconds=1)
        df = pd.concat([df, dftmp], axis=0, ignore_index=True, keys=None)
        
    df.to_csv('0y_eth_only17andnew.csv', index=False)
    
get_klines_iter('ETHBUSD', '5m', '2022-04-20', '2022-05-01')

In [3]:
df = pd.read_csv('0y_eth_only17andnew.csv')

In [4]:
df

,Date,Time,Open
0,2022/04/20,00:00:00,3102.26
1,2022/04/20,00:05:00,3095.42
2,2022/04/20,00:10:00,3098.42
3,2022/04/20,00:15:00,3105.58
4,2022/04/20,00:20:00,3102.21
...,...,...,...
3167,2022/04/30,23:40:00,2726.31
3168,2022/04/30,23:45:00,2725.49
3169,2022/04/30,23:50:00,2725.61
3170,2022/04/30,23:55:00,2728.93


In [5]:
df['date_time'] = df['Date'] + ' ' + df['Time']
df = df.drop(['Date', 'Time'],axis=1)
df['date_time'] = pd.to_datetime(df['date_time'])

In [6]:
df = df[['date_time', 'Open']]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3172 entries, 0 to 3171
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date_time  3172 non-null   datetime64[ns]
 1   Open       3172 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 49.7 KB


In [8]:
train_dates = pd.to_datetime(df['date_time'])

In [9]:
cols = list(df)[1:]

In [10]:
df_for_training = df[cols].astype(float)

In [11]:
scaler = StandardScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

In [12]:
features_train = []
target_train = []

In [13]:
n_future = 1 
n_past = 12

In [14]:
for i in range(n_past, len(df_for_training_scaled) - n_future + 1):
    features_train.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    target_train.append(df_for_training_scaled[i + n_future - 1:i + n_future, 0])

In [15]:
features_train, target_train = np.array(features_train), np.array(target_train)

In [16]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(features_train.shape[1], features_train.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(target_train.shape[1]))

In [17]:
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 12, 64)            16896     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 29,345
Trainable params: 29,345
Non-trainable params: 0
_________________________________________________________________


In [18]:
history = model.fit(features_train, target_train, epochs=200, batch_size=288, validation_split=0.1, verbose=1)

Epoch 1/200
10/10 [==============================] - 5s 122ms/step - loss: 0.7497 - val_loss: 1.1867
Epoch 2/200
10/10 [==============================] - 1s 65ms/step - loss: 0.4520 - val_loss: 0.0276
Epoch 3/200
10/10 [==============================] - 1s 65ms/step - loss: 0.1210 - val_loss: 0.0204
Epoch 4/200
10/10 [==============================] - 1s 66ms/step - loss: 0.0632 - val_loss: 0.0215
Epoch 5/200
10/10 [==============================] - 1s 66ms/step - loss: 0.0501 - val_loss: 0.0177
Epoch 6/200
10/10 [==============================] - 1s 69ms/step - loss: 0.0485 - val_loss: 0.0139
Epoch 7/200
10/10 [==============================] - 1s 70ms/step - loss: 0.0438 - val_loss: 0.0126
Epoch 8/200
10/10 [==============================] - 1s 70ms/step - loss: 0.0413 - val_loss: 0.0108
Epoch 9/200
10/10 [==============================] - 1s 68ms/step - loss: 0.0421 - val_loss: 0.0112
Epoch 10/200
10/10 [==============================] - 1s 65ms/step - loss: 0.0393 - val_loss: 0.009

In [20]:
n_past = 289
n_5min_for_prediction=288

In [21]:
predict_period_dates = pd.date_range(list(train_dates)[-n_past], periods=n_5min_for_prediction, freq='5min').tolist()

In [22]:
prediction = model.predict(features_train[-n_5min_for_prediction:])

9/9 [==============================] - 1s 8ms/step


In [23]:
prediction_copies = np.repeat(prediction, df_for_training.shape[1], axis=-1)
target_pred_future = scaler.inverse_transform(prediction_copies)[:,0]

In [24]:
forecast_dates = []
for time_i in predict_period_dates:
    forecast_dates.append(time_i)

In [25]:
df_forecast = pd.DataFrame({'date_time':np.array(forecast_dates), 'Open':target_pred_future})
df_forecast['date_time']=pd.to_datetime(df_forecast['date_time'])

In [26]:
orig = df[['date_time', 'Open']]

In [34]:
orig['date_time']=pd.to_datetime(orig['date_time'])
orig = orig.loc[orig['date_time'] >= '2022-04-30 00:05:00']

In [59]:
orig[orig.duplicated(['date_time'])].index

Int64Index([3000], dtype='int64')

In [61]:
orig = orig.drop(index=3000)

почему то образовался дубликат. Не время разбираться откуда он)

In [65]:
%matplotlib qt
plt.plot(orig['date_time'], orig['Open'])
plt.plot(df_forecast['date_time'], df_forecast['Open'])
plt.show()

In [69]:
mean_squared_error(orig['Open'], df_forecast['Open'])

45.69303986724091

Какой-то результат, c какой-то ошибкой

И вот мы получили результат, который даже можно назвать удовлетворительным, но вся проблема в попытках предсказать рынок заключается в том что мы не можем смоделировать финансовый рынок, по сути для нас, неимеющих обширных данных это просто рандомные скачки курса и ничего более, то что модели показывающие "приемлемые" на графиках результаты есть это правда, только почему то забываем о том что 99.99% этих моделей рисуют правильные предсказания, но на шаг вперед, а нам то нужно на шаг назад, нам нужно вчера знать что будет завтра, а не наоборот, так что как тестовое задание на понимание того что происходит это годится, а как что то, что можно в последствии доработать до жизнеспособности к сожалению нет. В целом создать трейд бота наверно можно, такого который на очень маленькой дистанции сможет давать правильные предсказания с достаточной точностью, но это уже отдельная тема для исследования. IMHO